# Equal-Weight S&P 500 Index Fund

##### The goal is to create a program in python that can take the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the fund

### Library Imports

In [1]:
import numpy as np
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
import xlsxwriter
import intrinio_sdk
import math

### Importing our Stocks (Tickers) from CSV

In [96]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


### Get the API Token from IEX Cloud

In [5]:
from secrets2 import API_TOKEN 

### API Call to get Market Capitalization and Price

In [108]:
# Construct the full URL with the necessary parameters
def getPrice(symbol, token):
    function = 'GLOBAL_QUOTE'
    base_url = 'https://www.alphavantage.co/query'
    url = f'{base_url}?function={function}&symbol={symbol}&apikey={api_key}'
    
    # Make the GET request
    response = requests.get(url)
    
    # Check the status code and print the result
    if response.status_code == 200:
        data = response.json()
        # The current stock price is in the '05. price' field of the 'Global Quote' section
        current_price = data['Global Quote']['05. price']
        return current_price
    else:
        print(f"Failed to retrieve data: {response.status_code}, {response.text}")

def getMarketCap(symbol, token):
    function = 'OVERVIEW'
    base_url = 'https://www.alphavantage.co/query'
    url = f'{base_url}?function={function}&symbol={symbol}&apikey={api_key}'
    
    # Make the GET request
    response = requests.get(url)
    
    # Check the status code and print the result
    if response.status_code == 200:
        data = response.json()
        # The current stock price is in the '05. price' field of the 'Global Quote' section
        current_price = data['MarketCapitalization']
        return current_price
    else:
        print(f"Failed to retrieve data: {response.status_code}, {response.text}")

In [112]:
price = getPrice("IBM", API_TOKEN)
market_cap = getMarketCap("IBM", API_TOKEN)

Current stock price of IBM: ${'Information': 'Thank you for using Alpha Vantage! Our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.'}


KeyError: 'MarketCapitalization'

### Add the Stock Data for Each stock into the Pandas DataFrame

In [90]:
import pandas as pd

# Define the columns
cols = ['ticker', 'stock price', 'market capitalization', 'number of shares to buy']

# Create an empty DataFrame with the specified columns
finalDataFrame = pd.DataFrame(columns=cols)

# Define the data to be added
symbol = 'AAPL'  # Example ticker

# Append the data using loc
finalDataFrame.loc[len(finalDataFrame)] = [symbol, price, market_cap, 'N/A']

# Display the DataFrame
print(finalDataFrame)


  ticker stock price market capitalization number of shares to buy
0   AAPL    177.3000          160847397000                     N/A


In [100]:
for stock in stocks['Ticker']:
    price = getPrice(stock, API_TOKEN)
    market_cap = getMarketCap(stock, API_TOKEN)
    finalDataFrame.loc[len(finalDataFrame)] = [stock, price, market_cap, 'N/A']

finalDataFrame

Current stock price of A: $125.7800
Current Market Cap of A: $37170352000
Current stock price of AAL: $11.0500
Current Market Cap of AAL: $7243233000
Current stock price of AAP: $60.6000
Current Market Cap of AAP: $3560047000
Current stock price of AAPL: $220.2700
Current Market Cap of AAPL: $3323666104000
Current stock price of ABBV: $165.9600
Current Market Cap of ABBV: $300851266000


KeyError: 'Global Quote'